In [80]:
# branch_df
# payment_df
# transaction_df
# product_df
# baskets_df

# save_branch_id = pd.DataFrame(transaction_df)
# save_branch_id.drop_duplicates(subset='branch', inplace=True)
# save_branch_id.to_csv('save_branch_id.csv', mode='a', header=False, index=False)
# save_branch_id = pd.read_csv('save_branch_id.csv')

import os
import numpy as np
import pandas as pd
import psycopg2
import psycopg2.extras as extras

# file_path = "reports_from_branches/chesterfield_25-08-2021_09-00-00.csv"
# file_path = "reports_from_branches/chesterfield_25-08-2021_09-00-00.csv"
# file_path = "reports_from_branches/chesterfield_12-01-2023_09-00-00.csv"
# file_path = "reports_from_branches/longridge_12-01-2023_09-00-00.csv"
file_path = "reports_from_branches/uppingham_12-01-2023_09-00-00.csv"
column_names = ["date_time", "branch", "customer", "order_content", "total_price", "payment_type", "credit_card_number"]
df = pd.read_csv(file_path, names=column_names)

def cleaning_and_arranging_df(df):
     
    #spliting the first time, turn order_content into a list
    df["order_content"]= df["order_content"].str.split(',')

    #spliting the order_content into individual row
    df = df.explode("order_content")

    #spliting from the last -, turn it into a list
    df["order_content"] = df["order_content"].str.rsplit(pat='-', n=1)

    #putting the two list items into two newly created columns
    df[['product_name','product_price']] = pd.DataFrame(df.order_content.tolist(), index= df.index)

    #striping whitespaces
    df['product_price'] = df['product_price'].str.strip()
    df['product_name'] = df['product_name'].str.strip()

    #drop old columns
    df = df.drop(columns=["order_content"])

    #rearrange columns
    new_col = ["date_time", "branch", "customer", "product_name","product_price", "total_price", "payment_type","credit_card_number"]
    df = df[new_col]

    #change the column to datetime format
    df["date_time"] = pd.to_datetime(df["date_time"])
    
    return df

def dropping_colums(df,drop_cols):
    df = df.drop(drop_cols, axis=1)
    return df

def create_foreign_key_dict(df, col_name):
    foreign_key_dict = {}
    for i, item in enumerate(df[col_name].unique()):
        foreign_key_dict[item] = i+1
    return foreign_key_dict

def add_foreign_key_column(df, new_col, exist_col, foreign_key_dict):
    df[new_col] = df[exist_col].map(foreign_key_dict)
    return df

branch_df = dropping_colums(df, ['date_time', 'order_content', 'total_price','payment_type','customer','credit_card_number'])
branch_df.rename(columns={'branch': 'branch_name'}, inplace=True)
branch_df = branch_df.drop_duplicates()

branch_df.to_csv('branch_id.csv', mode='a', index=False, header=False)
branch_df = pd.read_csv('branch_id.csv', names=["branch_name"])
branch_df = branch_df.drop_duplicates()


payment_df = dropping_colums(df, ['date_time','branch','order_content','total_price','customer','credit_card_number'])
payment_df = payment_df.drop_duplicates()

transaction_df = dropping_colums(df, ['order_content','customer','credit_card_number'])
transaction_df["date_time"] = pd.to_datetime(df["date_time"])

branch_dict = create_foreign_key_dict(transaction_df, 'branch')
payment_type_dict = create_foreign_key_dict(transaction_df, 'payment_type')

transaction_df = add_foreign_key_column(transaction_df, 'branch_id', 'branch', branch_dict)
transaction_df = add_foreign_key_column(transaction_df, 'payment_type_id', 'payment_type', payment_type_dict)
transaction_df = dropping_colums(transaction_df, ['branch', 'payment_type'])


df = cleaning_and_arranging_df(df)
df = dropping_colums(df, ['customer','credit_card_number'])
product_df = dropping_colums(df, ["date_time","branch","total_price", "payment_type"])
product_df = product_df.drop_duplicates()

df.index += 1
baskets_df = df.rename_axis('transaction_id').reset_index()


product_id_dict = create_foreign_key_dict(baskets_df, 'product_name')
baskets_df = add_foreign_key_column(baskets_df, 'product_id', 'product_name', product_id_dict)
baskets_df = dropping_colums(baskets_df, ['date_time', 'branch', 'product_name','product_price','total_price','payment_type'])
baskets_df = baskets_df.convert_dtypes()


branch_dict = {'Chesterfield': 1, 'Longridge': 2, 'Uppingham': 3}

payment_dict = {'Card': 1, 'Cash': 2}

product_dict = {'Regular Flavoured iced latte - Hazelnut': 1,
 'Large Latte': 2,
 'Large Flavoured iced latte - Caramel': 3,
 'Regular Flavoured iced latte - Caramel': 4,
 'Large Flavoured iced latte - Hazelnut': 5,
 'Regular Flavoured latte - Hazelnut': 6,
 'Large Flat white': 7,
 'Regular Latte': 8,
 'Regular Flat white': 9,
 'Large Flavoured latte - Hazelnut': 10,
 'Regular Flavoured iced latte - Vanilla': 11,
 'Large Flavoured iced latte - Vanilla': 12,
 'Large Speciality Tea - Earl Grey': 13,
 'Large Smoothies - Glowing Greens': 14,
 'Regular Speciality Tea - Earl Grey': 15,
 'Regular Mocha': 16,
 'Regular Smoothies - Glowing Greens': 17,
 'Large Glass of milk': 18,
 'Regular Glass of milk': 19,
 'Regular Frappes - Strawberries & Cream': 20,
 'Large Frappes - Strawberries & Cream': 21,
 'Large Mocha': 22,
 'Large Speciality Tea - Peppermint': 23,
 'Regular Speciality Tea - Peppermint': 24,
 'Regular Smoothies - Berry Beautiful': 25,
 'Regular Flavoured latte - Caramel': 26,
 'Large Smoothies - Berry Beautiful': 27,
 'Large Red Label tea': 28,
 'Large Flavoured hot chocolate - Vanilla': 29,
 'Regular Cortado': 30,
 'Large Hot chocolate': 31,
 'Large Speciality Tea - Green': 32,
 'Regular Hot chocolate': 33,
 'Regular Flavoured hot chocolate - Vanilla': 34}

product_df = pd.DataFrame.from_dict(product_dict, orient='index', columns=['Quantity'])
product_names = product_df.index
product_names.tolist()
product_names
product_df = pd.DataFrame(product_names)
product_df

# x4 = {
#  'Large Speciality Tea - Peppermint': 2,
#  'Regular Speciality Tea - Peppermint': 3,
#  'Regular Smoothies - Berry Beautiful': 4,
#  'Regular Flavoured latte - Caramel': 5,
#  'Large Mocha': 6,
#  'Regular Red Label tea': 7,
#  'Large Smoothies - Berry Beautiful': 8,
#  'Large Red Label tea': 9,
#  'Large Flavoured hot chocolate - Vanilla': 10,
#  'Regular Cortado': 11,
#  'Large Hot chocolate': 12,
#  'Large Flavoured latte - Hazelnut': 13,
#  'Regular Flavoured latte - Hazelnut': 14,
#  'Large Flavoured latte - Caramel': 15,
#  'Regular Speciality Tea - Green': 16,
#  'Large Cortado': 17,
#  'Large Speciality Tea - Green': 18,
#  'Regular Hot chocolate': 19,
#  'Regular Flavoured hot chocolate - Vanilla': 20}

# product_dict = {'Large Speciality Tea - Earl Grey': 1,
# 'Regular Mocha': 2,
# 'Regular Latte': 3,
# 'Large Smoothies - Glowing Greens': 4,
# 'Large Speciality Tea - Peppermint': 5,
# 'Regular Speciality Tea - Earl Grey': 6,
# 'Regular Speciality Tea - Peppermint': 7, 
# 'Regular Flavoured iced latte - Hazelnut': 8, 
# 'Regular Smoothies - Berry Beautiful': 9, 
# 'Large Flavoured iced latte - Caramel': 10, 
# 'Regular Flavoured latte - Caramel': 11, 
# 'Regular Smoothies - Glowing Greens': 12, 
# 'Large Mocha': 13, 
# 'Large Latte': 14, 
# 'Large Glass of milk': 15, 
# 'Regular Red Label tea': 16, 
# 'Regular Flavoured iced latte - Caramel': 17, 
# 'Regular Flat white': 18, 
# 'Large Smoothies - Berry Beautiful': 19, 
# 'Regular Flavoured iced latte - Vanilla': 20, 
# 'Regular Glass of milk': 21, 
# 'Large Red Label tea': 22, 
# 'Large Flat white': 23, 
# 'Regular Frappes - Strawberries & Cream': 24, 
# 'Large Flavoured hot chocolate - Vanilla': 25, 
# 'Large Flavoured iced latte - Vanilla': 26, 
# 'Large Frappes - Strawberries & Cream': 27, 
# 'Regular Cortado': 28,
# 'Large Flavoured iced latte - Hazelnut': 29,
# 'Large Hot chocolate': 30, 
# 'Large Flavoured latte - Hazelnut': 31, 
# 'Regular Flavoured latte - Hazelnut': 32, 
# 'Large Flavoured latte - Caramel': 33, 
# 'Regular Speciality Tea - Green': 34, 
# 'Large Cortado': 35, 
# 'Large Speciality Tea - Green': 36, 
# 'Regular Hot chocolate': 37, 
# 'Regular Flavoured hot chocolate - Vanilla': 38}

# xddf = pd.DataFrame.from_dict(product_dict, orient='index', columns=['d'])


# x1 = {'Regular Flavoured iced latte - Hazelnut': 1,
#  'Large Latte': 2,
#  'Large Flavoured iced latte - Caramel': 3,
#  'Regular Flavoured iced latte - Caramel': 4,
#  'Large Flavoured iced latte - Hazelnut': 5,
#  'Regular Flavoured latte - Hazelnut': 6,
#  'Large Flat white': 7,
#  'Regular Latte': 8,
#  'Regular Flat white': 9,
#  'Large Flavoured latte - Hazelnut': 10,
#  'Regular Flavoured iced latte - Vanilla': 11,
#  'Large Flavoured iced latte - Vanilla': 12}

# x2 = {

 
#  'Regular Flavoured iced latte - Hazelnut': 4,
#  'Large Flavoured latte - Hazelnut': 5,
#  'Regular Flavoured iced latte - Caramel': 6,
#  'Large Latte': 7,
#  'Regular Flat white': 8,
#  'Regular Flavoured iced latte - Vanilla': 9,
#  'Large Flat white': 10,
#  'Large Flavoured iced latte - Vanilla': 11,
#  'Large Flavoured iced latte - Hazelnut': 12}

# x3 = {'Large Speciality Tea - Earl Grey': 1,
#  'Large Smoothies - Glowing Greens': 2,
#  'Regular Speciality Tea - Earl Grey': 3,
#  'Regular Mocha': 4,
#  'Large Flavoured iced latte - Caramel': 5,
#  'Regular Smoothies - Glowing Greens': 6,
#  'Large Glass of milk': 7,
#  'Regular Flavoured iced latte - Caramel': 8,
#  'Regular Glass of milk': 9,
#  'Regular Frappes - Strawberries & Cream': 10,
#  'Large Frappes - Strawberries & Cream': 11,
#  'Large Mocha': 12}


# x4 = {'Regular Mocha': 1,
#  'Large Speciality Tea - Peppermint': 2,
#  'Regular Speciality Tea - Peppermint': 3,
#  'Regular Smoothies - Berry Beautiful': 4,
#  'Regular Flavoured latte - Caramel': 5,
#  'Large Mocha': 6,
#  'Regular Red Label tea': 7,
#  'Large Smoothies - Berry Beautiful': 8,
#  'Large Red Label tea': 9,
#  'Large Flavoured hot chocolate - Vanilla': 10,
#  'Regular Cortado': 11,
#  'Large Hot chocolate': 12,
#  'Large Flavoured latte - Hazelnut': 13,
#  'Regular Flavoured latte - Hazelnut': 14,
#  'Large Flavoured latte - Caramel': 15,
#  'Regular Speciality Tea - Green': 16,
#  'Large Cortado': 17,
#  'Large Speciality Tea - Green': 18,
#  'Regular Hot chocolate': 19,
#  'Regular Flavoured hot chocolate - Vanilla': 20}
# xddf


,0
0,Regular Flavoured iced latte - Hazelnut
1,Large Latte
2,Large Flavoured iced latte - Caramel
3,Regular Flavoured iced latte - Caramel
4,Large Flavoured iced latte - Hazelnut
5,Regular Flavoured latte - Hazelnut
6,Large Flat white
7,Regular Latte
8,Regular Flat white
9,Large Flavoured latte - Hazelnut
